# Titanic Top 4% with ensemble modeling

- 1. Introduction 소개
- 2. Load and check data 데이터 가져오기, 확인하기
    - 2.1 load data
    - 2.2 outlier detection
    - 2.3 joining train and test set
    - 2.4 check for null and missing values
- 3. Feature analysis 특성 분석
    - 3.1 Numerical values
    - 3.2 Categorical values
- 4. Filling missing values 값 채우기
    - 4.1 Age
- 5. Feature engineering 피쳐 엔지니어링
    - 5.1 Name/Title
    - 5.2 Family size
    - 5.3 cabin
    - 5.4 ticket
- 6. Modeling 모델링
    - 6.1 Simple modeling[
        - 6.1.1 Cross validate models 
        - 6.1.2 Hyperparameter tunning for best models
        - 6.1.3 plot learning curves](http://)
        - 6.1.4 Feature importance of the tree based classifiers
    - 6.2 Ensemble modeling
        - 6.2.1 Combining models
    - 6.3 Prediction
        - 6.3.1 Predict and Submit results

## 1. Introduction
- 특성 분석을 한 후 피쳐 엔지니어링 & 앙상블 모델링에 집중하자
- 타이타닉호의 생존율을 예측해보자

스크립트는 세부분으로 나뉜다
1. feature analysis
2. feature engineering
3. modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style='white', context='notebook', palette='deep')

import warnings 
warnings.filterwarnings(action='ignore')

# https://blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221817400937 여기서 에러 해결

## 2. Load and check data

### 2.1 Load data

In [2]:
train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
IDtest = test['PassengerId']

### 2.2 Outlier detection

In [3]:
# 피쳐의 데이터 프레임 df 사용하고 Tukey 방법에 따라 n개 이상의 outlier를 포함하는 관측치에 해당하는 인덱스 리스트를 반환한다

def detect_outliers(df, n, features):
    outlier_indices = []
    
    for col in features:
        # 1쿼터 (25%)
        Q1 = np.percentile(df[col], 25)
        # 3쿼터 (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range 사분위간 범위 (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # 피쳐 col에 대한 특이치 인덱스 리스트 결정
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        # col에 대해 발견된 특이치 인덱스를 특이치 인덱스 리스트에 추가
        outlier_indices.extend(outlier_list_col)
        # 세 개 이상의 특이치를 포함하는 관측치 선택
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(k for k, v in outlier_indices.items() if v > n)
        
    return multiple_outliers

# 연령, sibsp, parch, 요금에서 특이치 확인
Outliers_to_drop = detect_outliers(train, 2, ["Age", "SibSp", "Parch", "Fare"])

In [4]:
train.loc[Outliers_to_drop] # 특이치 행 표시

We detect 10 outliers. The 28, 89 and 342 passenger have an high Ticket Fare.
The 7 others have very high values of SibSP.
10개의 outlier를 탐색한다. 승객 28명, 89명, 342명은 티켓 요금이 비싸다.
나머지 7명은 sibsp의 값이 매우 높다.

In [5]:
# drop outliers
train = train.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

### 2.3 joining train and test set

In [6]:
# 범주형 변환 중 동일한 수의 특성을 얻을 수 있도록 train test set 결합
train_len = len(train)
dataset = pd.concat(objs=[train, test], axis=0).reset_index(drop=True)

I join train and test datasets to obtain the same number of features during categorical conversion (See feature engineering).

### 2.4 check for null and missing values

In [7]:
# Fill empty and NaNs values with NaN
dataset = dataset.fillna(np.nan)

# Check for Null values
dataset.isnull().sum()

Age and Cabin features have an important part of missing values.

Survived missing values correspond to the join testing dataset (Survived column doesn't exist in test set and has been replace by NaN values when concatenating the train and test set)

연령 및 객실 기능은 결측값에 중요한 부분을 차지한다

생존 결측값은 결합 테스트 데이터 집합에 해당한다(생존 열은 테스트 세트에 존재하지 않으며 열차와 테스트세트를 연결할 때 NaN 값으로 대체됨)

In [8]:
# infos
train.info()
train.isnull().sum()

In [9]:
train.head()

In [10]:
train.dtypes

In [11]:
## simmarize data
# Summarize and statistics
train.describe()

## 3. Feature analysis

### 3.1 Numerical values 숫자 값

In [12]:
# 숫자 값(SibSp Parch 나이 요금)과 생존자 사이의 상관 행렬
g = sns.heatmap(train[["Survived", "SibSp", "Parch", "Age", "Fare"]].corr(), annot=True, fmt=".2f", cmap="coolwarm")

Only Fare feature seems to have a significative correlation with the survival probability.

It doesn't mean that the other features are not usefull. Subpopulations in these features can be correlated with the survival. To determine this, we need to explore in detail these features

운임 기능만 생존 확률과 유의한 상관 관계가 있는 것으로 보입니다.

다른 기능들이 완전히 사용되지 않는다는 뜻은 아닙니다. 이러한 특징의 하위 모집단은 생존과 상관관계가 있을 수 있다. 이를 확인하기 위해서는 이러한 기능을 자세히 살펴볼 필요가 있습니다.

#### SibSp

In [13]:
# Explore SibSp feature vs Survived
g = sns.factorplot(x="SibSp", y="Survived", data=train, kind="bar", size=6, palette = "muted")
g.despine(left=True)
g = g.set_ylabels("survival probability")

#### Parch

In [14]:
# Explore Parch feature vs Survived
g = sns.factorplot(x="Parch", y="Survived", data=train, kind="bar", size=6, palette="muted")
g.despine(left=True)
g = g.set_ylabels("survival provavility")

Small families have more chance to survive, more than single (Parch 0), medium (Parch 3,4) and large families (Parch 5,6 ).

Be carefull there is an important standard deviation in the survival of passengers with 3 parents/children

작은 과는 단일 과(파치 0), 중간 과(파치 3,4) 및 큰 과(파치 5,6 )보다 생존 확률이 높다.

세 명의 부모/자녀가 있는 승객의 생존에 중요한 표준 편차가 있으므로 주의하십시오.

#### Age

In [15]:
# Explore Age vs Survived
g = sns.FacetGrid(train, col='Survived')
g = g.map(sns.distplot, "Age")

Age distribution seems to be a tailed distribution, maybe a gaussian distribution.

We notice that age distributions are not the same in the survived and not survived subpopulations. Indeed, there is a peak corresponding to young passengers, that have survived. We also see that passengers between 60-80 have less survived.

So, even if "Age" is not correlated with "Survived", we can see that there is age categories of passengers that of have more or less chance to survive.

It seems that very young passengers have more chance to survive.

연령 분포는 꼬리 분포로 보이며 가우스 분포일 수도 있습니다.

생존 하위 모집단과 생존 하위 모집단에서 연령 분포가 동일하지 않다는 것을 알 수 있습니다. 젊은 승객에 해당하는 봉우리가 있는데, 이들은 살아남았다. 우리는 또한 60-80세 사이의 승객들이 덜 살아남았다는 것을 볼 수 있다.

그래서, "나이"가 "생존"과 상관관계가 없다고 해도, 우리는 생존할 기회가 더 많거나 더 적은 승객들의 나이 범주가 있다는 것을 알 수 있다.

매우 어린 승객들은 생존할 수 있는 기회가 더 많아 보입니다.

In [16]:
# Explore Age distibution 
g = sns.kdeplot(train["Age"][(train["Survived"] == 0) & (train["Age"].notnull())], color="Red", shade = True)
g = sns.kdeplot(train["Age"][(train["Survived"] == 1) & (train["Age"].notnull())], ax = g, color="Blue", shade=True)
g.set_xlabel("Age")
g.set_ylabel("Frequency")
g = g.legend(["Not Survived", "Survived"])

When we superimpose the two densities , we cleary see a peak correponsing (between 0 and 5) to babies and very young childrens.
우리가 두 밀도를 겹칠 때, 우리는 아기들과 매우 어린 아이들에게 피크 코레폰(0에서 5 사이)을 보는 것을 분명히 한다.

#### Fare

In [17]:
dataset["Fare"].isnull().sum()

In [18]:
#Fill Fare missing values with the median value
dataset["Fare"] = dataset["Fare"].fillna(dataset["Fare"].median())

Since we have one missing value , i decided to fill it with the median value which will not have an important effect on the prediction.
결측값이 하나이므로 예측에 큰 영향을 미치지 않는 중위수로 채우기로 했다.

In [19]:
# Explore Fare distribution 
g = sns.distplot(dataset["Fare"], color="m", label="Skewness : %.2f"%(dataset["Fare"].skew()))
g = g.legend(loc="best")

As we can see, Fare distribution is very skewed. This can lead to overweigth very high values in the model, even if it is scaled.

In this case, it is better to transform it with the log function to reduce this skew.

보시다시피 요금 분포가 매우 왜곡되어 있습니다. 이렇게 하면 크기가 조정되더라도 모델에서 매우 높은 값이 나올 수 있습니다.

이 경우에는 로그 함수로 변환하여 이 왜곡을 줄이는 것이 좋습니다.

In [20]:
# Apply log to Fare to reduce skewness distribution
dataset["Fare"] = dataset["Fare"].map(lambda i: np.log(i) if i > 0 else 0)

In [21]:
g = sns.distplot(dataset["Fare"], color="b", label="Skewness : %.2f"%(dataset["Fare"].skew()))
g = g.legend(loc="best")

Skewness is clearly reduced after the log transformation
로그 변환 후 왜도가 확실히 감소합니다

### 3.2 Categorical values

#### sex

In [22]:
g = sns.barplot(x="Sex", y="Survived", data=train)
g = g.set_ylabel("Survival Probability")

In [23]:
train[["Sex", "Survived"]].groupby('Sex').mean()

It is clearly obvious that Male have less chance to survive than Female.

So Sex, might play an important role in the prediction of the survival.

For those who have seen the Titanic movie (1997), I am sure, we all remember this sentence during the evacuation : "Women and children first".

남성이 여성보다 생존 가능성이 적다는 것은 명백하다

그럼 성별은 생존을 예측하는 데 중요한 역할을 할 수도 있다

타이타닉 영화(1997)을 본 사람이라면 대피하는 동안 우리 모두는 이 문장을 기억할 것이다 "여성과 아이들이 먼저"

#### Pclass

In [24]:
# Explore Pclass vs Survived
g = sns.factorplot(x="Pclass", y="Survived", data=train, kind="bar", size=6, palette="muted")
g.despine(left=True)
g = g.set_ylabels("survival probability")

In [25]:
# Explore Pclass vs Survived by Sex
g = sns.factorplot(x="Pclass", y="Survived", hue="Sex", data=train, size=6, kind="bar", palette="muted")
g.despine(left=True)
g = g.set_ylabels("survival provavility")

The passenger survival is not the same in the 3 classes. First class passengers have more chance to survive than second class and third class passengers.

This trend is conserved when we look at both male and female passengers.

세 클래스에서 승객 생존율이 같지 않습니다. 일등석 승객은 이등석, 삼등석 승객보다 생존 가능성이 높다.

이러한 경향은 우리가 남성 승객과 여성 승객 모두를 볼 때 보존된다.

#### Embarked

In [26]:
dataset["Embarked"].isnull().sum()

In [27]:
#Fill Embarked nan values of dataset set with 'S' most frequent value
dataset["Embarked"] = dataset["Embarked"].fillna("S")

Since we have two missing values , i decided to fill them with the most fequent value of "Embarked" (S).

결측값이 두 개 있기 때문에, 저는 "Markered" (S)의 가장 큰 fequent 값으로 채우기로 했습니다.

In [28]:
# Explore Embarked vs Survived 
g = sns.factorplot(x="Embarked", y="Survived", data=train, size=6, kind="bar", palette="muted")
g.despine(left=True)
g = g.set_ylabels("survival probability")

It seems that passenger coming from Cherbourg (C) have more chance to survive.

My hypothesis is that the proportion of first class passengers is higher for those who came from Cherbourg than Queenstown (Q), Southampton (S).

Let's see the Pclass distribution vs Embarked

셰르부르(c)에서 오는 승객의 생존 기회가 더 많아진 것으로 보인다

나의 가설은 퀸스타운(Q), 사우샘프턴(s)보다 셰르부르에서 온 사람들의 일등석 승객 비율이 더 높다는 것이다.

Pclass 분포 vs Boarded 를 살펴보겠습니다

In [29]:
# Explore Pclass vs Embarked
g = sns.factorplot("Pclass", col="Embarked", data=train, size=6, kind="count", palette="muted")
g.despine(left=True)
g = g.set_ylabels("Count")

Indeed, the third class is the most frequent for passenger coming from Southampton (S) and Queenstown (Q), whereas Cherbourg passengers are mostly in first class which have the highest survival rate.

At this point, i can't explain why first class has an higher survival rate. My hypothesis is that first class passengers were prioritised during the evacuation due to their influence.

실제로, S와 Q에서 오는 승객은 3등급이 가장 많은 반면 c 승객은 생존율이 가장 높은 1등급이 대부분이다
이쯤에서 일등석이 생존율이 높은 이유는 설명할 수 없다. 내 가설은 대피하는 동안 일등석 승객들이 그들의 영향으로 인해 우선시 되었다는 것이다

## 4. Filling missing Values

### 4.1 Age
As we see, Age column contains 256 missing values in the whole dataset.

Since there is subpopulations that have more chance to survive (children for example), it is preferable to keep the age feature and to impute the missing values.

To adress this problem, i looked at the most correlated features with Age (Sex, Parch , Pclass and SibSP).

보시다시피, 나이 열에는 전체 데이터 집합의 결측값 256개가 포함되어 있다.
생존 기회가 더 많은 하위 집단(예: 어린이)이 존재하기 때문에 나이 특징을 유지하고 결측값을 귀속시키는 것이 바람직하다
이 문제를 해결하기 위해 에이지(Sex, Parch, Pclass, SibSp)와 가장 상관관계가 있는 특징을 살펴봤다

In [30]:
# Explore Age vs Sex, Parch, Pclass and SibSp
g = sns.factorplot(y="Age", x="Sex", data=dataset, kind="box")
g = sns.factorplot(y="Age", x="Sex", hue="Pclass", data=dataset, kind="box")
g = sns.factorplot(y="Age", x="Parch", data=dataset, kind="box")
g = sns.factorplot(y="Age", x="SibSp", data=dataset, kind="box")

Age distribution seems to be the same in Male and Female subpopulations, so Sex is not informative to predict Age.

However, 1rst class passengers are older than 2nd class passengers who are also older than 3rd class passengers.

Moreover, the more a passenger has parents/children the older he is and the more a passenger has siblings/spouses the younger he is.

남성과 여성의 하위 집단에서 연령 분포는 동일한 것으로 보여 성별은 나이를 예측하는 데 도움이 되지 않는다
그러나 1등석 승객은 2등석 승객보다 나이가 많고 3등석 승객보다 나이가 많다
게다가, 승객의 부모/자식을 더 많이 가질수록 나이가 더 많고 승객은 더 어린 형제/자매를 가지고 있다

In [31]:
# convert Sex into categorical value 0 for male and 1 for female
dataset["Sex"] = dataset["Sex"].map({"male": 0, "female": 1})

In [32]:
g = sns.heatmap(dataset[["Age", "Sex", "SibSp", "Parch", "Pclass"]].corr(), cmap="BrBG", annot=True)

The correlation map confirms the factorplots observations except for Parch. Age is not correlated with Sex, but is negatively correlated with Pclass, Parch and SibSp.

In the plot of Age in function of Parch, Age is growing with the number of parents / children. But the general correlation is negative.

So, i decided to use SibSP, Parch and Pclass in order to impute the missing ages.

The strategy is to fill Age with the median age of similar rows according to Pclass, Parch and SibSp.

상관관계맵은 parch를 제외한 팩터플롯 관측치를 확인합니다. 나이는 성과 상관관계가 없지만 Pclass, Parch, SibSp와 음의 상관관계가 있다
Parch의 함수 연령 그림에서 나이는 부모/자녀의 수에 따라 증가한다. 그러나 일반적인 상관관계는 음수이다
그래서 사라진 나이를 귀책하기 위해 sibsp, parch, pclass를 사용하기로 했다
이 전략은 pclass, parch, sibsp에 따라 유사한 행의 중위수로 Age를 채운다

In [33]:
# Filling missing value of Age

# Fill Age with the median age of similar rows according to Pclass, Parch and SibSp
# Index of Nan age rows
index_NaN_age = list(dataset["Age"][dataset["Age"].isnull()].index)

for i in index_NaN_age :
    age_med = dataset["Age"].median()
    age_pred = dataset["Age"][((dataset['SibSp'] == dataset.iloc[i]["SibSp"]) & (dataset['Parch'] == dataset.iloc[i]["Parch"]) & (dataset['Pclass'] == dataset.iloc[i]["Pclass"]))].median()
    if not np.isnan(age_pred) :
        dataset['Age'].iloc[i] = age_pred
    else :
        dataset['Age'].iloc[i] = age_med

In [34]:
g = sns.factorplot(x="Survived", y="Age", data=train, kind="box")
g = sns.factorplot(x="Survived", y="Age", data=train, kind="violin")

No difference between median value of age in survived and not survived subpopulation.

But in the violin plot of survived passengers, we still notice that very young passengers have higher survival rate.
연령의 중앙값과 하위 인구 사이의 차이는 없다. 하지만 생존한 승객들의 바이올린 줄거리에서는 여전히 매우 어린 승객들이 더 높은 생존율을 가지고 있다는 것을 알 수 있다

## 5. Feature engineering
### 5.1 Name/Title

In [35]:
dataset["Name"].head()

The Name feature contains information on passenger's title.

Since some passenger with distingused title may be preferred during the evacuation, it is interesting to add them to the model.
이름 기능에는 승객 직함에 대한 정보가 포함되어 있다
대피하는 동안 애매한 직함을 가진 승객들이 선호될 수 있기 때문에, 그들을 모델에 추가하는 것은 흥미롭다

In [36]:
# Get Title from Name
dataset_title = [i.split(",")[1].split(".")[0].strip() for i in dataset["Name"]]
dataset["Title"] = pd.Series(dataset_title)
dataset["Title"].head()

In [37]:
g = sns.countplot(x="Title", data=dataset)
g = plt.setp(g.get_xticklabels(), rotation=45)

There is 17 titles in the dataset, most of them are very rare and we can group them in 4 categories.
데이터 셋에는 17개의 제목이 있는데, 대부분은 매우 희귀하고 우리는 그것들을 4가지 범주로 분류할 수 있다

In [38]:
# Convert to categorical values Title
dataset["Title"] = dataset["Title"].replace(['Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
dataset["Title"] = dataset["Title"].map({"Master":0, "Miss":1, "Ms":1, "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
dataset["Title"] = dataset["Title"].astype(int)

In [39]:
g = sns.countplot(dataset["Title"])
g = g.set_xticklabels(["Master", "Miss/Ms/Mme/Mlle/Mrs", "Mr", "Rare"])

In [40]:
g = sns.factorplot(x="Title", y="Survived", data=dataset, kind="bar")
g = g.set_xticklabels(["Master", "Miss-Mrs", "Mr", "Rare"])
g = g.set_ylabels("survival probability")

"Women and children first"

It is interesting to note that passengers with rare title have more chance to survive.
여자와 아이가 먼저
희귀한 타이틀을 가진 승객들이 생존할 기회가 더 많다는 점이 흥미롭다

In [41]:
# Drop Name variable
dataset.drop(labels = ["Name"], axis = 1, inplace = True)

### 5.2 Family size

We can imagine that large families will have more difficulties to evacuate, looking for theirs sisters/brothers/parents during the evacuation. So, i choosed to create a "Fize" (family size) feature which is the sum of SibSp , Parch and 1 (including the passenger).

대가족이 대피하는 동안 자매, 형제, 부모를 찾아 대피하는데 어려움이 더 클 것으로 짐작할 수 있다. 그래서, 나는 sibsp, parch, 1(승객 포함)의 합인 "Fize"(가족 크기) 성을 만들기로 결정

In [42]:
# Create a family size descriptor from SibSp and Parch
dataset["Fsize"] = dataset["SibSp"] + dataset["Parch"] + 1

In [43]:
g = sns.factorplot(x="Fsize", y="Survived", data = dataset)
g = g.set_ylabels("Survival Probability")

The family size seems to play an important role, survival probability is worst for large families.

Additionally, i decided to created 4 categories of family size.
가족 규모가 중요한 역할을 하는 것 같은데, 대가족은 생존 확률이 가장 낮다
추가로 가족 크기를 4가지로 나누기로 했다

In [44]:
# Create new feature of family size
dataset['Single'] = dataset['Fsize'].map(lambda s: 1 if s == 1 else 0)
dataset['SmallF'] = dataset['Fsize'].map(lambda s: 1 if s == 2 else 0)
dataset['MedF'] = dataset['Fsize'].map(lambda s: 1 if 3 <= s <= 4 else 0)
dataset['LargeF'] = dataset['Fsize'].map(lambda s: 1 if s >= 5 else 0)

Factorplots of family size categories show that Small and Medium families have more chance to survive than single passenger and large families.

가족 크기 범주의 팩터플롯ㅇ르 보면 소형 및 중형 가족이 단일 승객 및 대가족보다 생존확률이 더 높음

In [46]:
# convert to indicator values Title and Embarked
dataset = pd.get_dummies(dataset, columns = ["Title"])
dataset = pd.get_dummies(dataset, columns = ["Embarked"], prefix="Em")

In [47]:
dataset.head() # 데이터 순서가 다르게 나옴

At this stage, we have 22 features.

이 단계에서는 22개의 특성이 있다

#### 5.3 Cabin

In [48]:
dataset["Cabin"].head()

In [49]:
dataset["Cabin"].describe() # top 내용도 다르게 나옴

In [50]:
dataset["Cabin"].isnull().sum()

The Cabin feature column contains 292 values and 1007 missing values.

I supposed that passengers without a cabin have a missing value displayed instead of the cabin number.
캐빈 피쳐 열에는 292개의 값과 1007개의 결측값이 포함되어 있다
객실 번호가 아니라 객실이 없는 승객에게 결측값이 표시되는 걸로 알고 있다

In [51]:
dataset["Cabin"][dataset["Cabin"].notnull()].head()

In [52]:
# Replace the Cabin number by the type of cabin 'X' if not
dataset["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in dataset['Cabin']])

The first letter of the cabin indicates the Desk, i choosed to keep this information only, since it indicates the probable location of the passenger in the Titanic.
오두막의 첫 글자는 데스크를 가리키고 있는데,이 정보가 타이타닉에 탑승한 승객의 가능한 위치를 가리키기 때문에 이 정보만 보관하도록 한다

In [53]:
g = sns.countplot(dataset["Cabin"], order=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'X'])

In [54]:
g = sns.factorplot(y="Survived", x="Cabin", data=dataset, kind="bar", order=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'X'])
g = g.set_ylabels("Survival Probability")

Because of the low number of passenger that have a cabin, survival probabilities have an important standard deviation and we can't distinguish between survival probability of passengers in the different desks.

But we can see that passengers with a cabin have generally more chance to survive than passengers without (X).

It is particularly true for cabin B, C, D, E and F.

객실을 가지고 있는 승객의 수가 적기 때문에 생존 확률은 중요한 표준 편차를 가지고 있고 우리는 다른 책상에 있는 승객의 생존 확률을 구별할 수 없습니다

그러나 우리는 일반적으로 객실을 가진 승객들이 (X)가 없는 승객들보다 더 많은 생존 기회를 갖는다는 것을 알 수 있다.

특히 객실 B, C, D, E, F에 해당된다

In [55]:
dataset = pd.get_dummies(dataset, columns = ["Cabin"], prefix="Cabin")

#### 5.4 Ticket

In [56]:
dataset["Ticket"].head()

It could mean that tickets sharing the same prefixes could be booked for cabins placed together. It could therefore lead to the actual placement of the cabins within the ship.

Tickets with same prefixes may have a similar class and survival.

So i decided to replace the Ticket feature column by the ticket prefixe. Which may be more informative.
같은 접두사를 가진 티켓이 함께 배치된 객실에 예약될 수 있다는 것을 의미, 따라서 배 안에 실제로 선실을 배치하는 것으로 이어질 수 있다

같은 접두사를 가진 티켓은 비슷한 등급과 생존을 가질 수 있다
그래서 티켓 피처 열을 티켓 접두사로 대체하기로 했다. 그게 더 유익할 수 있다


In [57]:
# Treat Ticket by extracting the ticket prefix. When there is no prefix it returns X

Ticket = []
for i in list(dataset.Ticket):
    if not i.isdigit(): 
        Ticket.append(i.replace(".", "").replace("/", "").strip().split(' ')[0]) # Take prefix
    else:
        Ticket.append("X")
        
dataset["Ticket"] = Ticket
dataset["Ticket"].head()

In [58]:
dataset = pd.get_dummies(dataset, columns = ["Ticket"], prefix="T")

In [59]:
# Create categorical values for Pclass
dataset["Pclass"] = dataset["Pclass"].astype("category")
dataset = pd.get_dummies(dataset, columns = ["Pclass"], prefix="Pc")

In [60]:
# Drop useless variables
dataset.drop(labels = ["PassengerId"], axis = 1, inplace = True)

In [61]:
dataset.head()

## 6. Modeling

In [62]:
# Separate train dataset and test dataset

train = dataset[:train_len]
test = dataset[train_len:]
test.drop(labels=["Survived"], axis = 1, inplace=True) # 에러가 몰까요

In [63]:
# Separate train features and label

train["Survived"] = train["Survived"].astype(int)

Y_train = train["Survived"]

X_train = train.drop(labels = ["Survived"], axis=1)

### 6.1 Simple modeling

#### 6.1.1 Cross validate models
I compared 10 popular clasifiers and evaluate the mean accuracy of each of them by a stratified kfold cross validation procedure.
인기있는 분류기 10개를 비교하고 계층화된 k폴드 교차 검증 절차를 통해 각각의 평균 정확도를 평가한다

- SVC
- Decision Tree
- AdaBoost
- Random Forest
- Extra Trees
- Gradient Boosting
- Multiple layer perceprton (neural network)
- KNN
- Logistic regression
- Linear Discriminant Analysis

In [64]:
# Cross validate model with kfold stratified cross val
kfold = StratifiedKFold(n_splits=10)

In [65]:
# Modeling step Test differents algorithms 
random_state = 2
classifiers = []
classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(LinearDiscriminantAnalysis())

cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier, X_train, y = Y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC","DecisionTree","AdaBoost","RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","KNeighboors","LogisticRegression","LinearDiscriminantAnalysis"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")

I decided to choose the SVC, AdaBoost, RandomForest , ExtraTrees and the GradientBoosting classifiers for the ensemble modeling.

앙상블 모델링을 위해 svc, AdaBoost, RandomForest, ExtraTree, GradientBoosting 분류기를 선택하기로 했다

#### 6.1.2 최적의 모델을 위한 하이퍼 파라미터 튜닝


I performed a grid search optimization for AdaBoost, ExtraTrees , RandomForest, GradientBoosting and SVC classifiers.

I set the "n_jobs" parameter to 4 since i have 4 cpu . The computation time is clearly reduced.

But be carefull, this step can take a long time, i took me 15 min in total on 4 cpu.

AdaBoost, ExtraTrees, RandomForest, GradientBoostion 및 Svc 분류기에 대한 그리드 검색 최적화를 수행했다.
CPU가 4개이므로 "n_jobs"매개 변수를 4로 설정한다. 계산 시간이 단축되었음
하지만 이 단계는 오래 걸릴 수 있으니 주의, 나는 4개의 CPU에서 총 15분이 걸렸다

In [66]:
# META MODELING  WITH ADABOOST, RF, EXTRATREES and GRADIENTBOOSTING

# AdaBoost
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"], "base_estimator__splitter" : ["best", "random"], "algorithm" : ["SAMME", "SAMME.R"], "n_estimators" :[1,2], "learning_rate":[0.0001, 0.001, 0.1, 0.2, 0.3, 1.5]}

gsadaDTC = GridSearchCV(adaDTC, param_grid = ada_param_grid, cv=kfold, scoring="accuracy", n_jobs=4, verbose=1)

gsadaDTC.fit(X_train, Y_train)

ada_best = gsadaDTC.best_estimator_

In [67]:
gsadaDTC.best_score_

In [68]:
# ExtraTrees
ExtC = ExtraTreesClassifier()

# Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
                "max_features": [1,3,10],
                "min_samples_split":[2,3,10],
                "min_samples_leaf":[1,3,10],
                "bootstrap":[False],
                "n_estimators":[100,300],
                "criterion":["gini"]}

gsExtC = GridSearchCV(ExtC, param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs=4, verbose = 1)
gsExtC.fit(X_train, Y_train)
ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

In [69]:
# RFC Parameters tunning
RFC = RandomForestClassifier()

# Search grid for optimal parameters
rf_param_grid = {"max_depth":[None],
                "max_features":[1,3,10],
                "min_samples_split":[2,3,10],
                "min_samples_leaf":[1,3,10],
                "bootstrap":[False],
                "n_estimators":[100,300],
                "criterion":["gini"]}

gsRFC = GridSearchCV(RFC, param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs=4, verbose=1)
gsRFC.fit(X_train, Y_train)
RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

In [70]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
                'n_estimators' : [100,200,300],
                'learning_rate' : [0.1, 0.05, 0.01],
                'max_depth' : [4,8],
                'min_samples_leaf': [100,150],
                'max_features' : [0.3, 0.1]
                }

gsGBC = GridSearchCV(GBC, param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs=4, verbose=1)
gsGBC.fit(X_train, Y_train)
GBC_best = gsGBC.best_estimator_

# best score
gsGBC.best_score_

In [71]:
# SVC classifier
SVMC = SVC(probability=True)
svc_param_grid = {'kernel' : ['rbf'],
                 'gamma': [0.001, 0.01, 0.1, 1],
                 'C' : [1,10,50,100,200,300,1000]}

gsSVMC = GridSearchCV(SVMC, param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs=4, verbose=1)
gsSVMC.fit(X_train, Y_train)
SVMC_best = gsSVMC.best_estimator_

# best score
gsSVMC.best_score_

#### 6.1.3 plot learning curves

Learning curves are a good way to see the overfitting effect on the training set and the effect of the training size on the accuracy.

학습 곡선은 훈련 세트에 대한 과적합 효과와 훈련 크기가 정확도에 미치는 영향을 확인할 수 있는 좋은 방법이다

In [72]:
# Generate a simple plot of the test and training learning curve
# 테스트 및 교육용 학습 곡선의 간단한 플롯 생성

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross_validation score")
    plt.legend(loc="best")
    return plt

g = plot_learning_curve(gsRFC.best_estimator_, "RF learning curves", X_train, Y_train, cv=kfold)
g = plot_learning_curve(gsExtC.best_estimator_, "ExtraTrees learning curves", X_train, Y_train, cv=kfold)
g = plot_learning_curve(gsSVMC.best_estimator_, "SVC learning curves", X_train, Y_train, cv=kfold)
g = plot_learning_curve(gsadaDTC.best_estimator_, "AdaBoost learning curves", X_train, Y_train, cv=kfold)
g = plot_learning_curve(gsGBC.best_estimator_, "GradientBoosting learning curves", X_train, Y_train, cv=kfold)

GradientBoosting and Adaboost classifiers tend to overfit the training set. According to the growing cross-validation curves GradientBoosting and Adaboost could perform better with more training examples.

SVC and ExtraTrees classifiers seem to better generalize the prediction since the training and cross-validation curves are close together.

gradientboosting 및 adaboost분류기는 훈련셋을 오버핏하는 경향이 있다. 증가하는 교차검증곡선에 따르면 그래디언트부스팅과 에이디부스트는 더 많은 훈련예시로 더 잘 수행할 수 있다

SVC와 EXtraTrees분류기는 훈련 및 교차 검증 곡선이 서로 가깝기 때문에 예측을 더 잘 일반화하는 것으로 보인다

#### 6.1.4 Feature importance of tree based classifiers

In order to see the most informative features for the prediction of passengers survival, i displayed the feature importance for the 4 tree based classifiers.

승객 생존 예측에 가장 유용한 특성을 보기위해 4가지 트리 기반 분류기에 대한 특성의 중요성을 표시했다

In [73]:
nrows = ncols = 2
fig, axes = plt.subplots(nrows = nrows, ncols = ncols, sharex="all", figsize=(15,15))

names_classifiers = [("AdaBoosting", ada_best), ("ExtraTrees", ExtC_best), ("RandomForest", RFC_best), ("GradientBoosting", GBC_best)]

nclassifier = 0
for row in range(nrows):
    for col in range(ncols):
        name = names_classifiers[nclassifier][0]
        classifier = names_classifiers[nclassifier][1]
        indices = np.argsort(classifier.feature_importances_)[::-1][:40]
        g = sns.barplot(y=X_train.columns[indices][:40], x=classifier.feature_importances_[indices][:40], orient='h', ax=axes[row][col])
        g.set_xlabel("Relative importance", fontsize=12)
        g.set_ylabel("Features", fontsize=9)
        g.tick_params(labelsize=9)
        g.set_title(name + "feature importance")
        nclassifier += 1

I plot the feature importance for the 4 tree based classifiers (Adaboost, ExtraTrees, RandomForest and GradientBoosting).

We note that the four classifiers have different top features according to the relative importance. It means that their predictions are not based on the same features. Nevertheless, they share some common important features for the classification , for example 'Fare', 'Title_2', 'Age' and 'Sex'.

Title_2 which indicates the Mrs/Mlle/Mme/Miss/Ms category is highly correlated with Sex.

We can say that:

- Pc_1, Pc_2, Pc_3 and Fare refer to the general social standing of passengers.

- Sex and Title_2 (Mrs/Mlle/Mme/Miss/Ms) and Title_3 (Mr) refer to the gender.

- Age and Title_1 (Master) refer to the age of passengers.

- Fsize, LargeF, MedF, Single refer to the size of the passenger family.

According to the feature importance of this 4 classifiers, the prediction of the survival seems to be more associated with the Age, the Sex, the family size and the social standing of the passengers more than the location in the boat.

네가지 트리기반 분류기에 대한 기능중요도를 그림으로 표시

ㅇ뤼는 네가지의 분류기가 상대적인 중요도에 따라 다른 상위 특징을 가지고 있다는 것을 주목한다. 두 사람의 예측이 같은 특성에 근거하지 않는다는 뜻이다. 그럼에도 불구하고 페어, 타이틀2, 나이, 성별 같은 분류에 공통적으로 중요한 특징을 공유한다

Ms/Mlle/Mme/Miss/Ms 범주가 Sex와 높은 상관관계를 가지고 있음을 나타내는 Title_2

우리는 이렇게 말할 수 있다.

Pc_1, Pc_2, Pc_3 및 요금은 승객의 일반적인 사회적 지위를 의미

성별 및 제목_2(Mrs/Mlle/Mme/Miss/Ms) 및 제목_3(Mr)은 성별을 나타냄

연령 및 제목_1 (주)는 승객의 나이를 의미함

Fsize, LargeF, MedF, Single은 승객 가족의 크기

이 4가지 분류기의 특징적 중요성에 따라 생존 예측은 배 안 위치보다 승객의 나이, 성별, 가족 규모, 사회적 지위와 더 관련이 있는 것으로 보인다.


In [74]:
test_Survived_RFC = pd.Series(RFC_best.predict(test), name="RFC")
test_Survived_ExtC = pd.Series(ExtC_best.predict(test), name="ExtC")
test_Survived_SVMC = pd.Series(SVMC_best.predict(test), name="SVC")
test_Survived_AdaC = pd.Series(ada_best.predict(test), name="Ada")
test_Survived_GBC = pd.Series(GBC_best.predict(test), name="GBC")

# Concatenate all classifier results
ensemble_results = pd.concat([test_Survived_RFC, test_Survived_ExtC, test_Survived_AdaC, test_Survived_GBC, test_Survived_SVMC], axis=1)

g = sns.heatmap(ensemble_results.corr(), annot=True)
# 결과가 조금씩 다름

The prediction seems to be quite similar for the 5 classifiers except when Adaboost is compared to the others classifiers.

The 5 classifiers give more or less the same prediction but there is some differences. Theses differences between the 5 classifier predictions are sufficient to consider an ensembling vote.

ada부스트를 다른 분류기와 비교할 때를 빼고는 5개의 분류기에 대한 예측은 상당히 유사한 것으로 보인다
5개의 분류기는 다소 같은 예측을 하지만 약간의 차이가 있다. 5개의 분류기 예측 사이의 이러한 차이는 앙상블 투표를 고려하기에 충분하다

### 6.2 Ensemble modeling
#### 6.2.1 Combining models
I choosed a voting classifier to combine the predictions coming from the 5 classifiers.

I preferred to pass the argument "soft" to the voting parameter to take into account the probability of each vote.

5개의 분류기에서 나오는 예측을 종합하기 위해 투표 분류기를 선택했다
각 투표의 확률을 고려하기 위해 "soft"라는 주장을 투표 매개변수에 전달하는 것을 선호했다? 먼솔


In [75]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best),
                                      ('svc', SVMC_best), ('adac', ada_best),
                                      ('gbc', GBC_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train, Y_train)

### 6.3 Prediction
#### 6.3.1 Predict and Submit results

In [76]:
test_Survived = pd.Series(votingC.predict(test), name="Survived")
results = pd.concat([IDtest, test_Survived], axis=1)
results.to_csv("ensemble_python_voting.csv", index=False)

끝 !!!!